In [1]:
!pip install -q keras==2.8.0
!pip install -q tensorflow==2.8.0
!pip install -q scikit-multiflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 2.8.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 27.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency 

In [2]:
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score,accuracy_score
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from multiprocessing.pool import ThreadPool
from sklearn.naive_bayes import GaussianNB
from collections import deque, defaultdict
from imblearn.over_sampling import SMOTE
from warnings import filterwarnings
from xgboost import XGBClassifier
from scipy.special import softmax
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from keras import backend as K
from typing import Callable
import tensorflow as tf
from sklearn import svm
from typing import List
import seaborn as sns
import pandas as pd
import numpy as np
import requests
import logging
import pickle
import random
import keras
import copy
import json
import sys
import os
import gc

logging.getLogger("tensorflow").setLevel(logging.ERROR)
sns.set(rc = {'figure.figsize':(22,12)}, style="whitegrid")
filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_path = '/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS/data'
code_path = '/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS/Codes/Shared Codes'
results_path = "/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS/Results"
feature_selection_results = '/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS/feature_selection_results'

In [5]:
sys.path.insert(0,code_path)
from genetic_programming import SymbolicRegressor
from binirizer import CustomLabelBinirizer
from ensemble import Ensemble, Classifier
from oselm import OSELMClassifier,set_use_know
from DynamicFeatureSelection import dynamic_feature_selection
from SharedFunctions import prepare_data,train_and_test,feature_evolving,save_pickle,load_pickle,save_object,load_object,generate_new_samples

In [6]:
very_bad_reward = -10
bad_reward = -5
accept_reward = 1

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# epsilon=0.97
# epsilon_decay=0.040
# # min_epsilon = 0.01
# episod = 200
# e = []
# for i in range(episod):
#     epsilon -= epsilon *  epsilon_decay
#     e.append(epsilon)
# print(epsilon)
# x = [ i for i in range(episod)]
# plt.plot(range(episod), e,label=f'epsilon decay policy')
# plt.xlabel('steps')
# plt.ylabel('epsilon')
# plt.savefig('0-25.svg', format='svg')
# plt.show()

In [7]:
def epsilon_greedy(expected_reward, epsilon=0.97) -> int:
    """
    expected_reward: list of expected rewards for each possible action
    epsilon: .
    """
    if np.random.rand() <= epsilon:
        return np.random.choice(list(range(len(expected_reward))))
    else:
        return np.argmax(expected_reward)
PolicyFunction  = Callable[[np.ndarray, float], int]

In [8]:
LEARNING_RATE = 0.001
def create_model(input_dim):
    K.clear_session()
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape=(input_dim,)))
    model.add(keras.layers.Dense(32, kernel_initializer='he_uniform', activation='relu'))
    model.add(keras.layers.Dense(16, kernel_initializer='he_uniform', activation='relu'))
    model.add(keras.layers.Dense(2,))
    model.compile(loss='mse', optimizer='adam')
    return model

In [9]:
class Agents:
    def __init__(self, evaluation_network,number_of_featuer, buffer_size: int = 800):

        self.evaluation_network = evaluation_network
        self.target_network = copy.deepcopy(self.evaluation_network)
        self.buffer_size = buffer_size
        self.fitted = False
        self.number_of_featuer = number_of_featuer
        self.reply_buffer = deque(maxlen=self.buffer_size)
        self.contrbution = np.random.rand()

    def make_action(self, curr_state: np.ndarray, policy_function: PolicyFunction, epsilon) -> int:
        if self.fitted:
            q_values = self.evaluation_network.predict(curr_state.reshape(-1, self.number_of_featuer),verbose=0)
            action = policy_function(q_values, epsilon)
        else:
            action = policy_function([0, 1], 1)
        return action

    def update_target_network(self):
        self.target_network = copy.deepcopy(self.evaluation_network)
        return

In [10]:
class AgentsSoftmax(Agents):
    agent_count = 0
    def __init__(self, evaluation_network,number_of_featuer, buffer_size=800):
        self.agent_id = AgentsSoftmax.agent_count
        AgentsSoftmax.agent_count += 1
        super().__init__(evaluation_network,number_of_featuer, buffer_size)

    def update_evaluation_network(self, batch_size=32, epochs=5, discount_factor=0.995):
        batch = random.sample(self.reply_buffer, batch_size)
        Q1, actions, Q2, rewards = [], [], [], []
        for transition in batch:
            Q1.append(transition[0])
            actions.append(transition[1])
            Q2.append(transition[2])
            rewards.append(transition[3])
        X_train = np.array(Q1).reshape(-1, self.number_of_featuer)
        expected_reward = self.evaluation_network.predict(np.array(Q1).reshape(-1, self.number_of_featuer),verbose=0)
        Q2 = self.target_network.predict(np.array(Q2).reshape(-1, self.number_of_featuer),verbose=0)
        for i, act in enumerate(actions[:-1]):
            expected_reward[i, act] = rewards[i] + (discount_factor * np.argmax(Q2[i]))
        y_train = expected_reward.copy()
        change_frequency = 0
        for state, next_state, reward, next_reward in zip(X_train[:-1], X_train[1:], rewards[: -1], rewards[1:]):
            if np.abs(state[self.agent_id] - next_state[self.agent_id]) == 1:
                self.contrbution += np.abs(reward - next_reward)
                change_frequency += 1
        self.contrbution = 0 if change_frequency==0 else self.contrbution/change_frequency
        self.evaluation_network.fit(X_train, y_train, epochs=epochs, verbose=0)
        self.fitted = True
        return

In [11]:
class AgentsRegression(Agents):
    agent_count = 0
    def __init__(self, evaluation_network,number_of_featuer, buffer_size=800):
        self.agent_id = AgentsRegression.agent_count
        AgentsRegression.agent_count += 1
        super().__init__(evaluation_network,number_of_featuer, buffer_size)

    def update_evaluation_network(self, batch_size=32, epochs=5, discount_factor=0.995):
        batch = random.sample(self.reply_buffer, batch_size)
        Q1, actions, Q2, rewards = [], [], [], []
        for transition in batch:
            Q1.append(transition[0])
            actions.append(transition[1])
            Q2.append(transition[2])
            rewards.append(transition[3])
        X_train = np.array(Q1).reshape(-1, self.number_of_featuer)

        expected_reward = self.evaluation_network.predict(np.array(Q1).reshape(-1, self.number_of_featuer),verbose=0)
        Q2 = self.target_network.predict(np.array(Q2).reshape(-1, self.number_of_featuer),verbose=0)
        for i, act in enumerate(actions[:-1]):
            expected_reward[i, act] = rewards[i] + (discount_factor * np.argmax(Q2[i]))
        y_train = expected_reward.copy()
        self.evaluation_network.fit(X_train, y_train, epochs=epochs, verbose=0)
        self.fitted = True
        return

In [12]:
class AgentsAverage(Agents):
    agent_count = 0
    def __init__(self, evaluation_network,number_of_featuer, buffer_size=800):
        self.agent_id = AgentsAverage.agent_count
        AgentsAverage.agent_count += 1
        super().__init__(evaluation_network,number_of_featuer, buffer_size)

    def update_evaluation_network(self, batch_size=32, epochs=5, discount_factor=0.995):
        batch = random.sample(self.reply_buffer, batch_size)
        Q1, actions, Q2, rewards = [], [], [], []
        for transition in batch:
            Q1.append(transition[0])
            actions.append(transition[1])
            Q2.append(transition[2])
            rewards.append(transition[3])
        X_train = np.array(Q1).reshape(-1, self.number_of_featuer)

        expected_reward = self.evaluation_network.predict(np.array(Q1).reshape(-1, self.number_of_featuer),verbose=0)
        Q2 = self.target_network.predict(np.array(Q2).reshape(-1, self.number_of_featuer),verbose=0)
        for i, act in enumerate(actions[:-1]):
            expected_reward[i, act] = rewards[i] + (discount_factor * np.argmax(Q2[i]))
        y_train = expected_reward.copy()
        WINDOW_SIZE = 4
        X_train_ = np.zeros((X_train.shape[0] // WINDOW_SIZE, X_train.shape[1]))
        y_train_ = []
        j = 0
        for i in range(0, batch_size, WINDOW_SIZE):
            window_of_states = X_train[i: i + WINDOW_SIZE].sum(axis=0) / WINDOW_SIZE
            window_of_rewards = sum(rewards[i: i + WINDOW_SIZE])
            r = window_of_rewards * window_of_states[self.agent_id]
            X_train_[j, :] = np.around(window_of_states)
            if window_of_states[self.agent_id] == 0:
                if window_of_rewards > 0.6:
                    r = window_of_rewards
                else:
                    r = window_of_rewards / WINDOW_SIZE
            y_train_.append(r)
            j += 1
        X_train = X_train_
        y_train = np.array(y_train_)
        self.evaluation_network.fit(X_train, y_train, epochs=epochs, verbose=0)
        self.fitted = True
        return

In [13]:
class AgentsSingle(Agents):
    agent_count = 0
    def __init__(self, evaluation_network,number_of_featuer, buffer_size=800):
        self.agent_id = AgentsSingle.agent_count
        AgentsSingle.agent_count += 1
        super().__init__(evaluation_network,number_of_featuer, buffer_size)

    def update_evaluation_network(self, batch_size=32, epochs=5, discount_factor=0.995):
        batch = random.sample(self.reply_buffer, batch_size)
        Q1, actions, Q2, rewards = [], [], [], []
        for transition in batch:
            Q1.append(transition[0])
            actions.append(transition[1])
            Q2.append(transition[2])
            rewards.append(transition[3])
        X_train = np.array(Q1).reshape(-1, self.number_of_featuer)
        expected_reward = self.evaluation_network.predict(np.array(Q1).reshape(-1, self.number_of_featuer),verbose=0)
        Q2 = self.target_network.predict(np.array(Q2).reshape(-1, self.number_of_featuer),verbose=0)
        for i, act in enumerate(actions[:-1]):
            expected_reward[i, act] = rewards[i] + (discount_factor * np.argmax(Q2[i]))
        y_train = expected_reward.copy()
        self.evaluation_network.fit(X_train, y_train, epochs=epochs, verbose=0)
        self.fitted = True
        return

In [14]:
def genetic_programming():
    return SymbolicRegressor(population_size=10,
            generations=5, stopping_criteria=0.85,
            p_crossover=0.7, p_subtree_mutation=0.1,
            p_hoist_mutation=0.05, p_point_mutation=0.1,
            max_samples=0.7, verbose=0,
            parsimony_coefficient=1e-4, random_state=42,
            function_set=['avg2', 'avg3', 'avg5','median3', 'median5', 'maximum2', 'maximum3', 'maximum5'],
            metric='f1-score')

In [15]:
def generate_oselm_models(number_of_hidden_neurons, apply_model_replacement=False):
    models= [OSELMClassifier(number_of_hidden_neurons, 'relu', binarizer=CustomLabelBinirizer(), random_state=42),
             OSELMClassifier(number_of_hidden_neurons, 'relu', binarizer=CustomLabelBinirizer(), random_state=42),
             OSELMClassifier(number_of_hidden_neurons, 'relu', binarizer=CustomLabelBinirizer(), random_state=42),
             OSELMClassifier(number_of_hidden_neurons, 'relu', binarizer=CustomLabelBinirizer(), random_state=42),
             ]

    ensemble = Ensemble(classifiers=models, program=genetic_programming(), apply_model_replacement=apply_model_replacement)
    return ensemble

def generate_ml_models(number_of_hidden_neurons, apply_model_replacement=False):
    models = [
              KNeighborsClassifier(5),
              KNeighborsClassifier(5),
              LogisticRegression(),
              LogisticRegression(),
              GaussianNB(),
              GaussianNB(),
              GaussianNB(),
              ]
    ensemble = Ensemble(classifiers=models, program=genetic_programming(), apply_model_replacement=apply_model_replacement)
    return ensemble

In [16]:
# test set percentage
TESTSIZE=0.2
def get_reward(X,Y ,subset_features,apply_model_replacement=True):
    if sum(subset_features)==0:return -10,-10
    global TESTSIZE
    subset_features = np.where(np.array(subset_features) == 1)[0]
    if subset_features.shape[0] == 0:return -5,-5
    X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=TESTSIZE)
    X_train, X_test = X_train[:,subset_features], X_test[:,subset_features]
    OSELM_Classifier = OSELMClassifier(X.shape[1]*3 // 2, 'relu', binarizer=CustomLabelBinirizer(), random_state=42)
    OSELM_Classifier.fit(X_train, y_train)
    y_pred1 = OSELM_Classifier.predict(X_test)
    acc = accuracy_score(y_test, y_pred1)
    reward = f1_score(y_test, y_pred1)
    return reward, acc

In [17]:
def reward_strategy(time_step: int, accuracy: float, accuracy_history: list, subset_features: list, error_rate: float, data_name:str, Chunck_Number: int,
                    beta: float = 0.99):

    folders= ["kpfaoselm_gp_combiner_with_model_replacement_evolving_random_forest","kpfaoselm_gp_combiner_with_model_replacement_evolving",
              "kpfaoselm_gp_combiner_with_model_replacement_random_forest","kpfaoselm_gp_combiner_with_model_replacement"]
    if sum(subset_features) == len(subset_features):
        return bad_reward,-accept_reward

    elif sum(subset_features) == 0:
        return very_bad_reward,-accept_reward

    elif accuracy > max(accuracy_history):
      random_fores_with_evolving = load_object(data_name+"_ensemble",os.path.join(os.path.join(results_path,folders[0]),data_name)).scores[Chunck_Number+1]["f1-score"]
      with_evolving = load_object(data_name+"_ensemble",os.path.join(os.path.join(results_path,folders[1]),data_name)).scores[Chunck_Number+1]["f1-score"]
      random_forest = load_object(data_name+"_ensemble",os.path.join(os.path.join(results_path,folders[2]),data_name)).scores[Chunck_Number+1]["f1-score"]
      without_evolving = load_object(data_name+"_ensemble",os.path.join(os.path.join(results_path,folders[3]),data_name)).scores[Chunck_Number+1]["f1-score"]
      if accuracy>=max([random_fores_with_evolving,with_evolving,random_forest,without_evolving]):
        return accuracy,accept_reward
      else:
        return accuracy/2,accept_reward

    elif accuracy < max(accuracy_history):
        return -accuracy,-accept_reward

    else:
        return -1 * (beta * error_rate + ((1 - beta) * (sum(subset_features) / len(subset_features)))),-1

In [18]:
def save_object(obj, filename,path):
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'wb') as outp:
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    outp.close()
def load_object(filename,path):
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'rb') as outp:
        loaded_object = pickle.load(outp)
    outp.close()
    return loaded_object

In [19]:
def softmax_distrbution(agents):
    contrbutions = []
    for agent in agents:
        contrbutions.append(agent.contrbution)
    return softmax(contrbutions)

def random_forest_distrbution(X_train,Y_train,num_of_agents, num_of_samples=1000):
    X,y = [],[]
    for i in range(num_of_samples):
        features_space = np.random.choice([0, 1], size=(num_of_agents,)).tolist()
        f1,accuracy = get_reward(X_train,Y_train, features_space)
        X.append(features_space)
        y.append(accuracy)
    X = np.array(X)
    y = np.array(y)
    rf = RandomForestRegressor(n_estimators=15)
    rf.fit(X, y)
    return rf.feature_importances_.tolist()

In [20]:
def prepare_data(csv_filename, target_column_name='class'):
    df = pd.read_csv(csv_filename)
    df = df.iloc[:80000, :]
    column_names = df.columns.tolist()
    if target_column_name not in column_names:
        target_column_name = column_names[-1]
    unique_vlaues = sorted(df[target_column_name].unique().tolist())
    df[target_column_name] = df[target_column_name].apply(lambda x: 0 if x == unique_vlaues[0] else 1)
    df[target_column_name] = df[target_column_name].astype('int')
    num_of_columns = len(column_names)
    df.columns = list(range(num_of_columns))
    return df

In [21]:
def feature_selection(algo_type, agents, X, Y, data_name, features_space=None, Chunck_Number=0, eposide=200):
    epsilon=0.97
    epsilon_decay=0.040
    NUMBER_OF_AGENTS = X.shape[1]
    best_features = []
    features_space = np.random.choice([0, 1], size=(NUMBER_OF_AGENTS,)).tolist() if features_space is None else features_space
    contrbutions = []
    if algo_type == 'random_forest':
        contrbutions = random_forest_distrbution(X,Y,NUMBER_OF_AGENTS)
    elif algo_type in ['single_agent', 'average']:
        contrbutions = [1] * NUMBER_OF_AGENTS
    for i in tqdm(range(eposide)):
        rewards = [0]
        next_feature_space = []
        if algo_type == 'softmax':
            contrbutions = softmax_distrbution(agents)
        for t in range(0, NUMBER_OF_AGENTS):
            action = agents[t].make_action(np.array(features_space.copy()), epsilon_greedy, epsilon)
            next_feature_space.append(action)
            if algo_type == 'single_agent':
                features_space[t] = action
        reward_as_F_Score,accuracy = get_reward(X,Y, next_feature_space)
        reward_at_time_t,best = reward_strategy(t, reward_as_F_Score, rewards, next_feature_space, 1 - accuracy, data_name, Chunck_Number)
        if best>0:
          best_features = next_feature_space
        rewards.append(reward_as_F_Score)
        total_reward = reward_at_time_t
        transition = []
        for t in range(0, NUMBER_OF_AGENTS):
            transition.clear()
            feature_space_copy = features_space.copy()
            transition.append(feature_space_copy)
            action = next_feature_space[t]
            transition.append(action)
            transition.append(next_feature_space)
            transition.append(total_reward * contrbutions[t])
            agents[t].reply_buffer.append(transition)
            if len(agents[t].reply_buffer) > 32 and i % 32 == 0:
                agents[t].update_evaluation_network()
        if i % 64 == 0:
            for agent in agents:
                if agent.fitted:
                    agent.update_target_network()
        epsilon -= epsilon *  epsilon_decay
        features_space = next_feature_space
    return best_features if len(best_features)>0 else next_feature_space

In [22]:
def generate_new_samples(buffer, y_values, n=500, y_col='label'):
    if not y_col in buffer.columns.tolist():
      y_col = buffer.columns.tolist()[-1]
    if y_values.sum() == 0:
       return buffer[buffer[y_col] == 1].sample(n, random_state=41)[:, :-1].values, np.array([1] * n)
    else:
      return buffer[buffer[y_col] == 0].sample(n,random_state=41)[:, :-1].values, np.array([0] * n)

In [23]:
def save_pickle(obj, file_name):
  with open(file_name, 'wb') as f:
    pickle.dump(obj, f)
def load_pickle(file_name):
  with open(file_name, 'rb') as f:
    d = pickle.load(f)
  return d

In [24]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report

def model_evaluation(X_train, X_test, y_train, y_test, selected_features):
    subset_features = np.where(np.array(selected_features) == 1)[0]
    if subset_features.shape[0] == 0:return 0
    X_train, X_test = X_train[:,subset_features], X_test[:,subset_features]
    model = OSELMClassifier(X_train.shape[1]*3 // 2, 'relu', binarizer=CustomLabelBinirizer(), random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    print(f1,recall,precision,accuracy)
    return f1,recall,precision,accuracy,y_pred

In [25]:
def dynamic_feature_selection(chunk_X, chunk_Y, X_test, y_test, save_path, data_name,
                              algorithm_type=['softmax','average','single_agent','random_forest'], chunk_number=0):
  AgentsSoftmax.agent_count,AgentsAverage.agent_count,AgentsSingle.agent_count,AgentsRegression.agent_count = 0,0,0,0
  softmax_agents,average_agents,single_agent_agents,random_forest_agents,result,over_all=[],[],[],[],[],[]
  NUM_OF_FEATURES = chunk_X.shape[1]
  for i in range(NUM_OF_FEATURES):

    if 'random_forest' in algorithm_type:
      random_forest_agents.append(AgentsSoftmax(create_model(NUM_OF_FEATURES),NUM_OF_FEATURES))
    if 'average' in algorithm_type:
      average_agents.append(AgentsAverage(create_model(NUM_OF_FEATURES),NUM_OF_FEATURES))
    if 'single_agent' in algorithm_type:
      single_agent_agents.append(AgentsSingle(create_model(NUM_OF_FEATURES),NUM_OF_FEATURES))
    if 'softmax' in algorithm_type:
      softmax_agents.append(AgentsRegression(create_model(NUM_OF_FEATURES),NUM_OF_FEATURES))


  softmax_results,average_results,single_agent_results,random_forest_results,voting_results = results_dic(save_path,chunk_number)

  if 'softmax' in algorithm_type:
    print("softmax")
    features_space = None if chunk_number==0  else load_object("softmax_mask_"+str(chunk_number-1),save_path)
    softmax_result = feature_selection('softmax', softmax_agents,chunk_X, chunk_Y, data_name, features_space, chunk_number)
    f1, recall ,precision ,accuracy ,y_predicte = model_evaluation(chunk_X, X_test, chunk_Y, y_test, softmax_result)
    over_all.append(f1)
    save_object(softmax_result, "softmax_mask_"+str(chunk_number),save_path)
    softmax_results["f1"].append(f1)
    softmax_results["recall"].append(recall)
    softmax_results["precision"].append(precision)
    softmax_results["accuracy"].append(accuracy)
    softmax_results["y_predicte"].append(y_predicte)
    save_object(softmax_results, "softmax_results",save_path)
    gc.collect()


  if 'average' in algorithm_type:
    print("average")
    features_space = None if chunk_number==0  else load_object("average_mask_"+str(chunk_number-1),save_path)
    average_result = feature_selection('average', average_agents,chunk_X, chunk_Y, data_name, features_space, chunk_number)
    f1, recall ,precision ,accuracy ,y_predicte = model_evaluation(chunk_X, X_test, chunk_Y, y_test, average_result)
    over_all.append(f1)
    save_object(average_result, "average_mask_"+str(chunk_number),save_path)
    average_results["f1"].append(f1)
    average_results["recall"].append(recall)
    average_results["precision"].append(precision)
    average_results["accuracy"].append(accuracy)
    average_results["y_predicte"].append(y_predicte)
    save_object(average_results, "average_results",save_path)
    gc.collect()

  if 'single_agent' in algorithm_type:
    print("single_agent")
    features_space = None if chunk_number==0  else load_object("single_agent_mask_"+str(chunk_number-1),save_path)
    single_agent_result = feature_selection('single_agent', single_agent_agents, chunk_X, chunk_Y, data_name, features_space, chunk_number)
    f1, recall ,precision  ,accuracy ,y_predicte = model_evaluation(chunk_X, X_test, chunk_Y, y_test, single_agent_result)
    over_all.append(f1)
    save_object(single_agent_result, "single_agent_mask_"+str(chunk_number),save_path)
    single_agent_results["f1"].append(f1)
    single_agent_results["recall"].append(recall)
    single_agent_results["precision"].append(precision)
    single_agent_results["accuracy"].append(accuracy)
    single_agent_results["y_predicte"].append(y_predicte)
    save_object(single_agent_results, "single_agent_results",save_path)
    gc.collect()

  if 'random_forest' in algorithm_type:
    print("random_forest")
    features_space = None if chunk_number==0  else load_object("random_forest_mask_"+str(chunk_number-1),save_path)
    random_forest_result = feature_selection('random_forest', random_forest_agents,chunk_X, chunk_Y, data_name, features_space, chunk_number)
    f1, recall ,precision  ,accuracy ,y_predicte = model_evaluation(chunk_X, X_test, chunk_Y, y_test, random_forest_result)
    over_all.append(f1)
    save_object(random_forest_result, "random_forest_mask_"+str(chunk_number),save_path)
    random_forest_results["f1"].append(f1)
    random_forest_results["recall"].append(recall)
    random_forest_results["precision"].append(precision)
    random_forest_results["accuracy"].append(accuracy)
    random_forest_results["y_predicte"].append(y_predicte)
    save_object(random_forest_results, "random_forest_results",save_path)
    gc.collect()

  softmax_result=load_object("softmax_mask_"+str(chunk_number),save_path)
  average_result=load_object("average_mask_"+str(chunk_number),save_path)
  single_agent_result=load_object("single_agent_mask_"+str(chunk_number),save_path)
  random_forest_result=load_object("random_forest_mask_"+str(chunk_number),save_path)

  for softmax,average,single,random in zip(softmax_result,average_result,single_agent_result,random_forest_result):
    sum_votes = sum([softmax,average,single,random])
    if sum_votes > (len(algorithm_type) // 2):result.append(1)
    elif sum_votes == (len(algorithm_type) // 2):
      rand = np.random.uniform(low=0,high=1)
      if rand >0.5:result.append(1)
      else:result.append(0)
    else:result.append(0)

  f1, recall ,precision  ,accuracy ,y_predicte = model_evaluation(chunk_X, X_test, chunk_Y, y_test, result)
  over_all.append(f1)
  save_object(result, "voting_mask_"+str(chunk_number),save_path)
  voting_results["f1"].append(f1)
  voting_results["recall"].append(recall)
  voting_results["precision"].append(precision)
  voting_results["accuracy"].append(accuracy)
  voting_results["y_predicte"].append(y_predicte)
  save_object(voting_results, "voting_results",save_path)
  gc.collect()

  all_masks = [softmax_result,average_result,single_agent_result,random_forest_result,result]
  all_f1 = [load_object("softmax_results",save_path)["f1"][chunk_number],
            load_object("average_results",save_path)["f1"][chunk_number],
            load_object("single_agent_results",save_path)["f1"][chunk_number],
            load_object("random_forest_results",save_path)["f1"][chunk_number],
            load_object("voting_results",save_path)["f1"][chunk_number]]
  best_mask = all_masks[all_f1.index(max(all_f1))]
  save_object(best_mask,"best_mask_"+str(chunk_number),save_path)

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
def random_forest_feature_selection(X, y):
    """
    return best feature from X using random forest
    """
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 4))
    sel.fit(X, y)
    return sel.get_support()

In [ ]:
# def main(f_name, data_name, train_size=0.80 , result_save_path="", ChunkNumber=0,
#          algorithm_type=['softmax','average','single_agent','random_forest'], chunk_size = 500):

#   datasets = {}
#   d = prepare_data(f_name)
#   d = d.sample(frac=1, random_state=42)
#   buffer = d.sample(n=5000)
#   d.reset_index(inplace=True)
#   d.replace([np.inf], 0, inplace=True)
#   datasets[f_name.split('/')[-1]] = d
#   results = {}
#   for key in tqdm(datasets.keys()):
#       is_first_in = False
#       results[key] = {'model_result': []}
#       data = datasets[key].values
#       X, Y = data[:, 0:-1], data[:, -1].astype('int')
#       number_of_chunk = X.shape[0]//chunk_size
#       print(number_of_chunk)
#       chunks_features = np.array_split(X, number_of_chunk)
#       chunks_labels = np.array_split(Y, number_of_chunk)
#       print("===================== dataset : {} ======================".format(key))
#       print(data.shape)
#       for CN,chunk_X, chunk_Y in tqdm(zip([*range(len(chunks_labels))],chunks_features, chunks_labels)):
#           if ChunkNumber != 0 and CN<ChunkNumber:continue
#           if is_first_in:algorithm_type=['softmax','average','single_agent','random_forest']
#           try:
#             chunk_X, chunk_Y = SMOTE().fit_resample(chunk_X, chunk_Y)
#           except:
#             if chunk_Y.sum() in [0, 1]:
#               new_samples, new_labels = generate_new_samples(buffer, chunk_Y)
#               chunk_X = np.concatenate((chunk_X, new_samples))
#               chunk_Y = np.concatenate((chunk_Y, new_labels))
#           gc.collect()
#           X_train, X_test, y_train, y_test = train_test_split(chunk_X, chunk_Y, random_state=42, train_size=train_size)
#           dynamic_feature_selection(chunk_X=X_train, chunk_Y=y_train, X_test=X_test, y_test=y_test,
#                                     save_path=result_save_path, data_name=data_name, chunk_number=CN, algorithm_type=algorithm_type)

#           RF_selected = random_forest_feature_selection(X_train, y_train)
#           f1, recall ,precision  ,accuracy ,y_predicte = model_evaluation(chunk_X, X_test, chunk_Y, y_test, RF_selected)
#           is_first_in = True
#           print("CHUNK NUMBER ",CN,"DONE")

In [96]:
def main(f_name, data_name, train_size=0.80 , result_save_path="", ChunkNumber=0,
         algorithm_type=['softmax','average','single_agent','random_forest'], chunk_size = 500):

  datasets = {}
  d = prepare_data(f_name)
  d = d.sample(frac=1, random_state=42)
  buffer = d.sample(n=5000)
  d.reset_index(inplace=True)
  d.replace([np.inf], 0, inplace=True)
  datasets[f_name.split('/')[-1]] = d
  results = {}
  for key in tqdm(datasets.keys()):
      is_first_in = False
      results[key] = {'model_result': []}
      data = datasets[key].values
      X, Y = data[:, 0:-1], data[:, -1].astype('int')
      number_of_chunk = X.shape[0]//chunk_size
      chunks_features = np.array_split(X, number_of_chunk)
      chunks_labels = np.array_split(Y, number_of_chunk)
      all_chunck = []
      for CN,chunk_X, chunk_Y in tqdm(zip([*range(len(chunks_labels))],chunks_features, chunks_labels)):
          chunk_Y = np.reshape(chunk_Y, (chunk_Y.shape[0], 1))
          data_array = np.hstack((chunk_X, chunk_Y))
          chunk_number_array = np.array(chunk_X.shape[0]*[CN])
          # data_array = np.hstack((data_array,np.reshape(chunk_number_array, (chunk_number_array.shape[0], 1))))
          data_frame = pd.DataFrame(data_array)
          data_frame['chunk_number'] = chunk_number_array
          all_chunck.append(data_frame)
  concatenated_df = pd.concat(all_chunck, axis=0)
  concatenated_df.to_csv(os.path.join('/'.join(f_name.split('/')[:-1]),"modified_"+data_name), index=False)

In [97]:
def results_dic(path,chunk_number=0):
  if chunk_number == 0 :
    softmax_results = {"f1":[],"recall":[],"precision":[],"accuracy":[],"y_predicte":[]}
    average_results = {"f1":[],"recall":[],"precision":[],"accuracy":[],"y_predicte":[]}
    single_agent_results = {"f1":[],"recall":[],"precision":[],"accuracy":[],"y_predicte":[]}
    random_forest_results = {"f1":[],"recall":[],"precision":[],"accuracy":[],"y_predicte":[]}
    voting_results = {"f1":[],"recall":[],"precision":[],"accuracy":[],"y_predicte":[]}
  else:
    softmax_results = load_object('softmax_results',path)
    average_results = load_object('average_results',path)
    single_agent_results = load_object('single_agent_results',path)
    random_forest_results = load_object('random_forest_results',path)
    voting_results = load_object('voting_results',path)
  return softmax_results,average_results,single_agent_results,random_forest_results,voting_results

In [98]:
filenames = ['kddcup99_csv.csv','ISCX2012.csv','CSE-CIC2018.csv','CICIDS2017.csv','Recurrent_Drift.csv', 'Blip_Drift.csv', 'Incremental_Drift.csv',
             'Gradual_Drift.csv', 'Sudden_Drift.csv']
filenames = list(map(lambda x: os.path.join(data_path, x), filenames))

In [99]:
data_name = ['kddcup99','ISCX2012','CSE-CIC2018','CICIDS2017','Recurrent_Drift', 'Blip_Drift', 'Incremental_Drift','Gradual_Drift', 'Sudden_Drift']

In [100]:
# # for f_name,d_name in zip(filenames,data_name):
# #   path = os.path.join(feature_selection_results_evolving,d_name)
# #   os.makedirs(path, exist_ok=True)
# #   main(f_name, d_name, result_save_path=path)

In [101]:
DATA_NUMBER=0
ChunkNumber=97
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
path = os.path.join(feature_selection_results,d_name)
os.makedirs(path, exist_ok=True)
main(f_name, 'kddcup99_csv.csv', result_save_path=path, ChunkNumber=ChunkNumber,
  algorithm_type=['softmax','average','single_agent','random_forest'])

  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [102]:
DATA_NUMBER=1
ChunkNumber=0
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
path = os.path.join(feature_selection_results,d_name)
os.makedirs(path, exist_ok=True)
main(f_name, 'ISCX2012.csv', result_save_path=path, ChunkNumber=ChunkNumber,chunk_size=1000,
     algorithm_type=['single_agent','random_forest'])

  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [103]:
 DATA_NUMBER=2
 ChunkNumber=0
 f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
 path = os.path.join(feature_selection_results,d_name)
 os.makedirs(path, exist_ok=True)
 main(f_name, 'CSE-CIC2018.csv', result_save_path=path, ChunkNumber=ChunkNumber,
     algorithm_type=['softmax','average','single_agent','random_forest'], chunk_size=3500)

  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [104]:
DATA_NUMBER=3
ChunkNumber=0
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
path = os.path.join(feature_selection_results,d_name)
os.makedirs(path, exist_ok=True)
main(f_name, 'CICIDS2017.csv', result_save_path=path, ChunkNumber=ChunkNumber,
  algorithm_type=['random_forest'], chunk_size=1000)

  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [ ]:
# DATA_NUMBER=4
# ChunkNumber=87
# f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
# path = os.path.join(feature_selection_results,d_name)
# os.makedirs(path, exist_ok=True)
# main(f_name, 'Recurrent_Drift.csv', result_save_path=path, ChunkNumber=ChunkNumber,
# algorithm_type=['softmax','average','single_agent','random_forest'])

In [ ]:
# DATA_NUMBER=5
# ChunkNumber=93
# f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
# path = os.path.join(feature_selection_results,d_name)
# os.makedirs(path, exist_ok=True)
# main(f_name, 'Blip_Drift.csv', result_save_path=path, ChunkNumber=ChunkNumber,
# algorithm_type=['average','single_agent','random_forest'])

In [ ]:
# DATA_NUMBER=6
# ChunkNumber=89
# f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
# path = os.path.join(feature_selection_results,d_name)
# os.makedirs(path, exist_ok=True)
# main(f_name, 'Incremental_Drift.csv', result_save_path=path, ChunkNumber=ChunkNumber,
# algorithm_type=['single_agent','random_forest'])

In [ ]:
# DATA_NUMBER=7
# ChunkNumber=0
# f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
# path = os.path.join(feature_selection_results,d_name)
# os.makedirs(path, exist_ok=True)
# main(f_name, 'Gradual_Drift.csv', result_save_path=path, ChunkNumber=ChunkNumber)

In [ ]:
# DATA_NUMBER=8
# ChunkNumber=98
# f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
# path = os.path.join(feature_selection_results,d_name)
# os.makedirs(path, exist_ok=True)
# main(f_name, 'Sudden_Drift.csv', result_save_path=path, ChunkNumber=ChunkNumber,
#     algorithm_type=['random_forest'])